# GenAI Tracing and Evaluation with MLflow 3 

https://mlflow.org/docs/latest/genai/data-model/experiments 


In [1]:
%reload_ext autoreload
%autoreload 2

import getpass
import os
import sys
from pathlib import Path

import openai


### Secrets and Environment Variables

MLflow:<br>
`MLFLOW_TRACKING_URI`<br>

OpenAI API Key:<br>
`OPENAI_API_KEY`


In [2]:
os.environ["MLFLOW_TRACKING_URI"] = "http://0.0.0.0:5001"
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPEN API key:")

Enter OPEN API key: ········


### Check connection to MLflow


In [3]:
import mlflow 

# List experiments in MLflow
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/501136849596978774', creation_time=1750315227989, experiment_id='501136849596978774', last_update_time=1750315227989, lifecycle_stage='active', name='6-question-answering-evaluation', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1750143170502, experiment_id='0', last_update_time=1750143170502, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
# Set up MLflow experiment
mlflow.set_experiment("5-genai-with-mlflow-3")

2025/06/19 13:48:34 INFO mlflow.tracking.fluent: Experiment with name '5-genai-with-mlflow-3' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/329902449898594710', creation_time=1750315714873, experiment_id='329902449898594710', last_update_time=1750315714873, lifecycle_stage='active', name='5-genai-with-mlflow-3', tags={}>

# Example 1: Tracing a GenAI Application



https://mlflow.org/docs/latest/genai/getting-started/tracing/tracing-notebook 

## Automatic Tracing of LLM calls

In [5]:
# Enable MLflow's autologging to instrument your application with Tracing
mlflow.openai.autolog()

# Create an OpenAI client
client = openai.OpenAI()


# Use the trace decorator to capture the application's entry point
@mlflow.trace
def my_app(input: str):
    # This call is automatically instrumented by `mlflow.openai.autolog()`
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": input,
            },
        ],
    )
    return response.choices[0].message.content


my_app(input="What is MLflow?")

'MLflow is an open-source platform designed to manage the machine learning lifecycle. It provides a set of tools to help data scientists and machine learning engineers track experiments, reproduce results, and deploy models. MLflow has four main components:\n\n1. **MLflow Tracking**: This component allows users to log and query experiments. It helps in tracking parameters, metrics, artifacts, and even the source code used to generate the model. This makes it easier to compare different experiments and reproduce results.\n\n2. **MLflow Projects**: This component offers a way to package data science code in a reusable and reproducible manner. Projects can be defined using a simple YAML format that specifies dependencies, parameters, and entry points. This helps in sharing code and reproducing the same environment for experimentation.\n\n3. **MLflow Models**: MLflow Models provide a standard format for packaging machine learning models, allowing them to be deployed to various platforms (s

Trace(trace_id=62a92698c6874be1841b98df6ea72c79)

## Tracing LangChain🦜⛓️

https://mlflow.org/docs/latest/genai/tracing/integrations/listing/langchain 


In [7]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# Enabling autolog for LangChain will enable trace logging.
mlflow.langchain.autolog()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, max_tokens=1000)

prompt_template = PromptTemplate.from_template(
    "Answer the question as if you are {person}, fully embodying their style, wit, personality, and habits of speech. "
    "Emulate their quirks and mannerisms to the best of your ability, embracing their traits—even if they aren't entirely "
    "constructive or inoffensive. The question is: {question}"
)

chain = prompt_template | llm | StrOutputParser()

# Let's test another call
chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

"Oh, come on. Seriously? Look, I get it—you want to make things easier, but giving everyone sudo access is like handing out loaded guns at a kindergarten. It might seem convenient for a while, but I can assure you, chaos will ensue. \n\nSure, you might think, “Ah, no more permissions issues! Hooray!” But then, the first time someone accidentally deletes a critical system file or decides to play around with something they shouldn’t, you’ll be sitting there wondering why you ever thought this was a good idea.\n\nThe whole point of using sudo and managing access is to maintain some level of control and security. You want to empower users, sure, but you also want to keep the ship afloat. So, instead of going full-on “let them all do whatever,” how about educating them on what they can do and why it matters? \n\nYou know, it’s not just about making things easier; it's about making sure that the system doesn't implode because someone thought it would be fun to run `rm -rf /` just because the

Trace(trace_id=50688b2601614096ae7f2669a5ed434e)

## Token Usage Tracking

https://mlflow.org/docs/latest/genai/tracing/integrations/listing/langchain#token-usage-tracking


In [8]:
# Execute the chain defined in the previous example
chain.invoke(
    {
        "person": "Linus Torvalds",
        "question": "Can I just set everyone's access to sudo to make things easier?",
    }
)

# Get the trace object just created
last_trace_id = mlflow.get_last_active_trace_id()
trace = mlflow.get_trace(trace_id=last_trace_id)

# Print the token usage
total_usage = trace.info.token_usage
print("== Total token usage: ==")
print(f"  Input tokens: {total_usage['input_tokens']}")
print(f"  Output tokens: {total_usage['output_tokens']}")
print(f"  Total tokens: {total_usage['total_tokens']}")

# Print the token usage for each LLM call
print("\n== Token usage for each LLM call: ==")
for span in trace.data.spans:
    if usage := span.get_attribute("mlflow.chat.tokenUsage"):
        print(f"{span.name}:")
        print(f"  Input tokens: {usage['input_tokens']}")
        print(f"  Output tokens: {usage['output_tokens']}")
        print(f"  Total tokens: {usage['total_tokens']}")

== Total token usage: ==
  Input tokens: 81
  Output tokens: 243
  Total tokens: 324

== Token usage for each LLM call: ==
ChatOpenAI:
  Input tokens: 81
  Output tokens: 243
  Total tokens: 324
Completions:
  Input tokens: 81
  Output tokens: 243
  Total tokens: 324


Trace(trace_id=1a14b1b2d4ba418b874c489769220794)

# Example 2:  Tracing LangGraph🦜🕸️

https://mlflow.org/docs/latest/genai/tracing/integrations/listing/langgraph 


In [9]:
from typing import Literal

import mlflow

from langchain_core.messages import AIMessage, ToolCall
from langchain_core.outputs import ChatGeneration, ChatResult
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# Enabling tracing for LangGraph (LangChain)
mlflow.langchain.autolog()


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"


llm = ChatOpenAI(model="gpt-4o-mini")
tools = [get_weather]
graph = create_react_agent(llm, tools)

# Invoke the graph
result = graph.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf?"}]}
)

Trace(trace_id=2377bfe7e7074e8097e7695a8eba771a)

# Example 3: Prompt Management

https://mlflow.org/docs/latest/genai/mlflow-3/genai-agent

In [10]:

system_prompt = mlflow.genai.register_prompt(
    name="chatbot_prompt",
    template="You are a chatbot that can answer questions about IT. Answer this question: {{question}}",
    commit_message="Initial version of chatbot",
)

2025/06/19 13:50:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: chatbot_prompt, version 1


In [11]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(system_prompt.to_single_brace_format())
chain = prompt | ChatOpenAI(temperature=0.7) | StrOutputParser()
question = "What is MLflow?"
print(chain.invoke({"question": question}))
# MLflow is an open-source platform for managing the end-to-end machine learning lifecycle...

MLflow is an open-source platform for managing the end-to-end machine learning lifecycle. It provides tools for tracking experiments, packaging code into reproducible runs, and sharing and deploying models. MLflow supports multiple machine learning frameworks and languages, making it easier for data scientists and engineers to collaborate and manage their machine learning projects.


Trace(trace_id=d11af9ad9a9b4ce8b41e849b956d9c83)

In [13]:
# set the active model for linking traces
# mlflow.set_active_model(name="langchain_model")

# Enable autologging so that interactive traces from the client are automatically linked to a LoggedModel
mlflow.langchain.autolog()

questions = [
    "What is MLflow Tracking and how does it work?",
    "What is Unity Catalog?",
    "What are user-defined functions (UDFs)?",
]
outputs = []

for question in questions:
    outputs.append(chain.invoke({"question": question}))

# fetch the current active model's id and check traces
active_model_id = mlflow.get_active_model_id()
mlflow.search_traces(model_id=active_model_id)
#                            trace_id                                             trace  ...  assessments                        request_id
# 0  e807ab0a020f4794989a24c84c2892ad  Trace(trace_id=e807ab0a020f4794989a24c84c2892ad)  ...           []  e807ab0a020f4794989a24c84c2892ad
# 1  4eb83e4adb6a4f3494bc5b33aca4e970  Trace(trace_id=4eb83e4adb6a4f3494bc5b33aca4e970)  ...           []  4eb83e4adb6a4f3494bc5b33aca4e970
# 2  42b100851f934c969c352930f699308d  Trace(trace_id=42b100851f934c969c352930f699308d)  ...           []  42b100851f934c969c352930f699308d

,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,868c3db3225b49b8892ecfbefcce676a,Trace(trace_id=868c3db3225b49b8892ecfbefcce676a),None,TraceState.OK,1750315833639,1285,{'question': 'What are user-defined functions ...,User-defined functions (UDFs) are functions th...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'OxGM6pzYzqEv/zZu4UOi4Q==', 'spa...",[]
1,cf9719be06f649f683e5915c4c435a4b,Trace(trace_id=cf9719be06f649f683e5915c4c435a4b),None,TraceState.OK,1750315832094,1519,{'question': 'What is Unity Catalog?'},Unity Catalog is a tool used in Unity's Asset ...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': '9NOkFjG7MKtlsd4VTJVvsQ==', 'spa...",[]
2,199d207ceafc4bfe85c5a2fbe90bfd2d,Trace(trace_id=199d207ceafc4bfe85c5a2fbe90bfd2d),None,TraceState.OK,1750315830235,1824,{'question': 'What is MLflow Tracking and how ...,MLflow Tracking is a component of the MLflow o...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'z8iGeO0z8WP+Nbq9f1DOKw==', 'spa...",[]
3,1da508bd9c464aaeaee1f05734ae5982,Trace(trace_id=1da508bd9c464aaeaee1f05734ae5982),None,TraceState.OK,1750315818674,1172,{'question': 'What are user-defined functions ...,User-defined functions (UDFs) are functions th...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'QlXFmt/uxJ9wM0r3j/ypaQ==', 'spa...",[]
4,454a931391f441a2a688faa34b79f100,Trace(trace_id=454a931391f441a2a688faa34b79f100),None,TraceState.OK,1750315816412,2228,{'question': 'What is Unity Catalog?'},Unity Catalog is a feature within the Unity ga...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'IE5ywvQflMs8RWlDkUD6Nw==', 'spa...",[]
5,2f5de3df42384661b29a864109794c03,Trace(trace_id=2f5de3df42384661b29a864109794c03),None,TraceState.OK,1750315814164,2232,{'question': 'What is MLflow Tracking and how ...,MLflow Tracking is a component of the MLflow o...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': '03xlLf0wQAaIhSnbMx7Amg==', 'spa...",[]
6,d11af9ad9a9b4ce8b41e849b956d9c83,Trace(trace_id=d11af9ad9a9b4ce8b41e849b956d9c83),None,TraceState.OK,1750315810602,981,{'question': 'What is MLflow?'},MLflow is an open-source platform for managing...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': '/WripYWvK4XOLokuu6ca6A==', 'spa...",[]
7,2377bfe7e7074e8097e7695a8eba771a,Trace(trace_id=2377bfe7e7074e8097e7695a8eba771a),None,TraceState.OK,1750315799445,1677,"{'messages': [{'role': 'user', 'content': 'wha...",{'messages': [{'content': 'what is the weather...,"{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': '4sskc0hjM6V/i4SzLqlxWw==', 'spa...",[]
8,1a14b1b2d4ba418b874c489769220794,Trace(trace_id=1a14b1b2d4ba418b874c489769220794),None,TraceState.OK,1750315792489,4366,"{'person': 'Linus Torvalds', 'question': 'Can ...","Oh, come on! That's like handing out candy to ...","{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': 'XuXv2hHSPDk2a3r4hiO55A==', 'spa...",[]
9,50688b2601614096ae7f2669a5ed434e,Trace(trace_id=50688b2601614096ae7f2669a5ed434e),None,TraceState.OK,1750315785117,3930,"{'person': 'Linus Torvalds', 'question': 'Can ...","Oh, come on. Seriously? Look, I get it—you wan...","{'mlflow.trace.tokenUsage': '{""input_tokens"": ...",{'mlflow.artifactLocation': 'mlflow-artifacts:...,"[{'trace_id': '3sWrydOclERrKYvGD7i1Jg==', 'spa...",[]


[Trace(trace_id=199d207ceafc4bfe85c5a2fbe90bfd2d), Trace(trace_id=cf9719be06f649f683e5915c4c435a4b), Trace(trace_id=868c3db3225b49b8892ecfbefcce676a)]

# Example 4:Evaluate the agent's performance

In [14]:
# Prepare the eval dataset in a pandas DataFrame
import pandas as pd

eval_df = pd.DataFrame(
    {
        "messages": questions,
        "expected_response": [
            """MLflow Tracking is a key component of the MLflow platform designed to record and manage machine learning experiments. It enables data scientists and engineers to log parameters, code versions, metrics, and artifacts in a systematic way, facilitating experiment tracking and reproducibility.\n\nHow It Works:\n\nAt the heart of MLflow Tracking is the concept of a run, which is an execution of a machine learning code. Each run can log the following:\n\nParameters: Input variables or hyperparameters used in the model (e.g., learning rate, number of trees). Metrics: Quantitative measures to evaluate the model's performance (e.g., accuracy, loss). Artifacts: Output files like models, datasets, or images generated during the run. Source Code: The version of the code or Git commit hash used. These logs are stored in a tracking server, which can be set up locally or on a remote server. The tracking server uses a backend storage (like a database or file system) to keep a record of all runs and their associated data.\n\n Users interact with MLflow Tracking through its APIs available in multiple languages (Python, R, Java, etc.). By invoking these APIs in the code, you can start and end runs, and log data as the experiment progresses. Additionally, MLflow offers autologging capabilities for popular machine learning libraries, automatically capturing relevant parameters and metrics without manual code changes.\n\nThe logged data can be visualized using the MLflow UI, a web-based interface that displays all experiments and runs. This UI allows you to compare runs side-by-side, filter results, and analyze performance metrics over time. It aids in identifying the best models and understanding the impact of different parameters.\n\nBy providing a structured way to record experiments, MLflow Tracking enhances collaboration among team members, ensures transparency, and makes it easier to reproduce results. It integrates seamlessly with other MLflow components like Projects and Model Registry, offering a comprehensive solution for managing the machine learning lifecycle.""",
            """Unity Catalog is a feature in Databricks that allows you to create a centralized inventory of your data assets, such as tables, views, and functions, and share them across different teams and projects. It enables easy discovery, collaboration, and reuse of data assets within your organization.\n\nWith Unity Catalog, you can:\n\n1. Create a single source of truth for your data assets: Unity Catalog acts as a central repository of all your data assets, making it easier to find and access the data you need.\n2. Improve collaboration: By providing a shared inventory of data assets, Unity Catalog enables data scientists, engineers, and other stakeholders to collaborate more effectively.\n3. Foster reuse of data assets: Unity Catalog encourages the reuse of existing data assets, reducing the need to create new assets from scratch and improving overall efficiency.\n4. Enhance data governance: Unity Catalog provides a clear view of data assets, enabling better data governance and compliance.\n\nUnity Catalog is particularly useful in large organizations where data is scattered across different teams, projects, and environments. It helps create a unified view of data assets, making it easier to work with data across different teams and projects.""",
            """User-defined functions (UDFs) in the context of Databricks and Apache Spark are custom functions that you can create to perform specific tasks on your data. These functions are written in a programming language such as Python, Java, Scala, or SQL, and can be used to extend the built-in functionality of Spark.\n\nUDFs can be used to perform complex data transformations, data cleaning, or to apply custom business logic to your data. Once defined, UDFs can be invoked in SQL queries or in DataFrame transformations, allowing you to reuse your custom logic across multiple queries and applications.\n\nTo use UDFs in Databricks, you first need to define them in a supported programming language, and then register them with the SparkSession. Once registered, UDFs can be used in SQL queries or DataFrame transformations like any other built-in function.\n\nHere\'s an example of how to define and register a UDF in Python:\n\n```python\nfrom pyspark.sql.functions import udf\nfrom pyspark.sql.types import IntegerType\n\n# Define the UDF function\ndef multiply_by_two(value):\n    return value * 2\n\n# Register the UDF with the SparkSession\nmultiply_udf = udf(multiply_by_two, IntegerType())\n\n# Use the UDF in a DataFrame transformation\ndata = spark.range(10)\nresult = data.withColumn("multiplied", multiply_udf(data.id))\nresult.show()\n```\n\nIn this example, we define a UDF called `multiply_by_two` that multiplies a given value by two. We then register this UDF with the SparkSession using the `udf` function, and use it in a DataFrame transformation to multiply the `id` column of a DataFrame by two.""",
        ],
        "predictions": outputs,
    }
)

# Start a run to represent the evaluation job
with mlflow.start_run() as evaluation_run:
    eval_dataset = mlflow.data.from_pandas(
        df=eval_df,
        name="eval_dataset",
        targets="expected_response",
        predictions="predictions",
    )
    mlflow.log_input(dataset=eval_dataset)
    # Run the evaluation based on extra metrics
    # Current active model will be automatically used
    result = mlflow.evaluate(
        data=eval_dataset,
        extra_metrics=[
            mlflow.metrics.genai.answer_correctness("openai:/gpt-4o"),
            mlflow.metrics.genai.answer_relevance("openai:/gpt-4o"),
        ],
        # This is needed since answer_correctness looks for 'inputs' field
        evaluator_config={"col_mapping": {"inputs": "messages"}},
    )

result.tables["eval_results_table"]
#                                         messages  ...                  answer_relevance/v1/justification
# 0  What is MLflow Tracking and how does it work?  ...  The output directly addresses the input questi...
# 1                         What is Unity Catalog?  ...  The output is completely irrelevant to the inp...
# 2        What are user-defined functions (UDFs)?  ...  The output directly addresses the input questi...

2025/06/19 13:50:40 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
/Users/artemsokolov/.pyenv/versions/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:03<00:00,  1.27s/it]


🏃 View run placid-colt-462 at: http://0.0.0.0:5001/#/experiments/329902449898594710/runs/80fc517e96bf4f1e92859cdc34402656
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/329902449898594710


,messages,expected_response,predictions,answer_correctness/v1/score,answer_correctness/v1/justification,answer_relevance/v1/score,answer_relevance/v1/justification
0,What is MLflow Tracking and how does it work?,MLflow Tracking is a key component of the MLfl...,MLflow Tracking is a component of the MLflow o...,5,The output is correct and demonstrates a high ...,5,The output directly addresses the input questi...
1,What is Unity Catalog?,Unity Catalog is a feature in Databricks that ...,Unity Catalog is a tool used in Unity's Asset ...,1,The output is completely incorrect as it descr...,1,The output is completely irrelevant to the inp...
2,What are user-defined functions (UDFs)?,User-defined functions (UDFs) in the context o...,User-defined functions (UDFs) are functions th...,3,The output addresses the basic concept of user...,5,The output directly addresses the input questi...
